# Unificar los bomberos y su distribucion territorial

### Importación de librerías

In [1]:
import pandas as pd
import os
import re

In [ ]:
os.listdir(f"./dataset/variables_alternas/")

['DIRECCIONES_Y_TELEFONOS_MSPAS_1ER_SEMESTRE 2025.xlsx',
 'DIRECCIONES_CBM.pdf',
 'DIRECCIONES_BOMBEROS_DEPARTAMENTALES.pdf',
 'DIRECCIONES_BOMBEROS_DEPARTAMENTALES.xlsx',
 'DIAGRAMA_OCUPACIONES.pdf',
 'Distribución_Territorial_de_las_instalaciones_de_servicios_de_justicia.xlsx']

## Distribución de bomberos departamentales

In [ ]:
df = pd.read_excel(f"./dataset/variables_alternas/DIRECCIONES_BOMBEROS_DEPARTAMENTALES.xlsx",
                   dtype=str,
                   header=9,
                   index_col=0)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 133 entries, 2 to 134
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 1    133 non-null    object
 1   Unnamed: 2    0 non-null      object
 2   DIRECCIÓN.1   133 non-null    object
 3   DEPARTAMENTO  133 non-null    object
 4   TELÉFONO      133 non-null    object
 5   CORREO        132 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [5]:
df = df.drop(columns='Unnamed: 2') # Borrar columna invalida
df = df.rename(columns={
    "Unnamed: 1": "Municipio"
}) # Renombrar la columna por municipio

In [6]:
df.columns

Index(['Municipio', 'DIRECCIÓN.1', 'DEPARTAMENTO', 'TELÉFONO', 'CORREO'], dtype='object')

In [7]:
df.head()

,Municipio,DIRECCIÓN.1,DEPARTAMENTO,TELÉFONO,CORREO
DIRECCIÓN,,,,,
2,Jocotenango,1era. Avenida 7-00 Colonia Los\nLlanos,Sacatepéquez,79221961 - 40279096,bomberosdejocotenango@gmail.com
3,Chichicastenango,2da. Calle entre 4ta. Y 5ta. Av.\nKm. 220,Quiche,40350823,bomberosmunicipaleschichi@hotmail.com
4,Coatepeque,2da.Av. 1-12 Zona 3 Barrio San Francisco,Quetzaltenango,77751122 - 40277335 -\n77750638,coatepequebomberos@gmail.com
5,Camanchaj,Km.121 ruta internacional 15,Quiche,40308567,bomberoscamanchaj@gmail.com
6,San Pedro Sac. S.M.,3ra. Av. C 3-77 zona 4 canón El Mosquito San ...,San Marcos,40230150 - 77602176,bomberosmunicipalessps@hotmail.es


### Settear los departamentos de Guatemala

In [8]:
# Lista oficial de departamentos de Guatemala
departamentos_guatemala = [
    "Alta Verapaz",
    "Baja Verapaz",
    "Chimaltenango",
    "Chiquimula",
    "El Progreso",
    "Escuintla",
    "Guatemala",
    "Huehuetenango",
    "Izabal",
    "Jalapa",
    "Jutiapa",
    "Peten",
    "Quetzaltenango",
    "Quiche",
    "Retalhuleu",
    "Sacatepequez",
    "San Marcos",
    "Santa Rosa",
    "Solola",
    "Suchitepequez",
    "Totonicapan",
    "Zacapa"
]

### Limpiar los departamentos

In [9]:
import re
import unicodedata
from difflib import get_close_matches

# Function to remove accents and normalize text
def remove_accents(text):
    text = unicodedata.normalize('NFKD', text)
    return ''.join([c for c in text if not unicodedata.combining(c)])

# Basic regex-based cleaning
def clean_departament(depto, depto_oficiales):
    depto = remove_accents(depto)
    depto = re.sub(r'\s+', ' ', depto).strip()
    depto_oficiales = [remove_accents(d) for d in depto_oficiales]
    if depto in depto_oficiales:
        return depto
    for depto_ in depto_oficiales:
        if re.search(depto_, depto, re.IGNORECASE):
            return depto_
    return None

# Fuzzy matching with accent removal
def clean_departamento(depto, deptos_oficiales):
    depto = remove_accents(depto)
    deptos_oficiales = [remove_accents(d) for d in deptos_oficiales]
    match = get_close_matches(depto, deptos_oficiales, n=1, cutoff=0.6)
    return match[0] if match else depto

df["DEPARTAMENTO_LIMPIO"] = df["DEPARTAMENTO"].apply(lambda x: clean_departamento(str(x), departamentos_guatemala))

In [10]:
df["DEPARTAMENTO_LIMPIO"].value_counts()

DEPARTAMENTO_LIMPIO
San Marcos        19
Chimaltenango     13
Sacatepequez      12
Guatemala         12
Suchitepequez     10
Quiche             9
Jutiapa            9
Santa Rosa         8
Alta Verapaz       7
Quetzaltenango     7
Zacapa             4
El Progreso        4
Peten              3
Escuintla          3
Totonicapan        2
Solola             2
Jalapa             2
Retalhuleu         2
San Lucas Sac.     1
Huehuetenango      1
Chiquimula         1
Mixco              1
Izabal             1
Name: count, dtype: int64

In [11]:
print(df[df["DEPARTAMENTO_LIMPIO"].isnull()].shape)
df[df["DEPARTAMENTO_LIMPIO"].isnull()]

(0, 6)


,Municipio,DIRECCIÓN.1,DEPARTAMENTO,TELÉFONO,CORREO,DEPARTAMENTO_LIMPIO
DIRECCIÓN,,,,,,


In [12]:
df['DEPARTAMENTO'] = df['DEPARTAMENTO'].replace(
    {
        'Mixco':'Guatemala',
        'San Lucas Sac.':'Sacatepequez', 
        'Masagua, Escuintla':'Escuintla',
        'Moyuta, Jutiapa':'Jutiapa',
        'San Pablo San Marcos':'San Marcos',
        'San Pablo, San Marcos':'San Marcos',
    }
)
df["DEPARTAMENTO_LIMPIO"] = df["DEPARTAMENTO"].apply(lambda x: clean_departamento(str(x), departamentos_guatemala))

In [13]:
df["DEPARTAMENTO_LIMPIO"].value_counts()

DEPARTAMENTO_LIMPIO
San Marcos        19
Sacatepequez      13
Chimaltenango     13
Guatemala         13
Suchitepequez     10
Quiche             9
Jutiapa            9
Santa Rosa         8
Quetzaltenango     7
Alta Verapaz       7
El Progreso        4
Zacapa             4
Escuintla          3
Peten              3
Totonicapan        2
Jalapa             2
Solola             2
Retalhuleu         2
Huehuetenango      1
Chiquimula         1
Izabal             1
Name: count, dtype: int64

### Convertir a los nombres oficiales de departamentos de guatemala

In [14]:
# Lista oficial de departamentos de Guatemala
departamentos_guatemala_real = {
    "Alta Verapaz":"Alta Verapaz",
    "Baja Verapaz":"Baja Verapaz",
    "Chimaltenango":"Chimaltenango",
    "Chiquimula":"Chiquimula",
    "El Progreso":"El Progreso",
    "Escuintla":"Escuintla",
    "Guatemala":"Guatemala",
    "Huehuetenango":"Huehuetenango",
    "Izabal":"Izabal",
    "Jalapa":"Jalapa",
    "Jutiapa":"Jutiapa",
    "Peten":"Petén",
    "Quetzaltenango":"Quetzaltenango",
    "Quiche":"Quiché",
    "Retalhuleu":"Retalhuleu",
    "Sacatepequez":"Sacatepéquez",
    "San Marcos":"San Marcos",
    "Santa Rosa":"Santa Rosa",
    "Solola":"Sololá",
    "Suchitepequez":"Suchitepéquez",
    "Totonicapan":"Totonicapán",
    "Zacapa":"Zacapa"
}
df['DEPARTAMENTO_LIMPIO'] = df['DEPARTAMENTO_LIMPIO'].replace(departamentos_guatemala_real)

### Mapear a los municipios de los diferentes departamentos

In [15]:
municipios_departamentos = [
    ("San José el Golfo", "Guatemala"),
    ("San Raimundo", "Guatemala"),
    ("Petapa", "Guatemala"),
    ("Guatemala", "Guatemala"),
    ("Patulul", "Suchitepéquez"),
    ("Alotenango", "Sacatepéquez"),
    ("Comalapa", "Chimaltenango"),
    ("Yepocapa", "Chimaltenango"),
    ("San Rafael las Flores", "Santa Rosa"),
    ("San Lucas Toliman", "Sololá"),
    ("San Cristobal Totonicapán", "Totonicapán"),
    ("Santa Lucía La Reforma", "Totonicapán"),
    ("Ostuncalco", "Quetzaltenango"),
    ("Olintepeque", "Quetzaltenango"),
    ("Colomba", "Quetzaltenango"),
    ("Palestina de los Altos", "Quetzaltenango"),
    ("San José El Idolo", "Suchitepéquez"),
    ("San Pablo Jocopilas", "Suchitepéquez"),
    ("Sololá", "Sololá"),
    ("Samayac", "Suchitepéquez"),
    ("Rio Bravo", "Escuintla"),
    ("Mixco", "Guatemala"),
    ("San Rafael Pié de La Cuesta", "San Marcos"),
    ("El Rodeo", "San Marcos"),
    ("Ocós", "San Marcos"),
    ("Ixtahuacán", "Huehuetenango"),
    ("Unión Cantinil", "Huehuetenango"),
    ("Chichicastenango", "Quiché"),
    ("Uspantán", "Quiché"),
    ("Ixcán", "Quiché"),
    ("Momostenango", "Totonicapán"),
    ("Nebaj", "Quiché"),
    ("Tucurú", "Alta Verapaz"),
    ("Lanquín", "Alta Verapaz"),
    ("Cahabón", "Alta Verapaz"),
    ("Santa Catalina la Tinta", "Alta Verapaz"),
    ("Jocotán", "Chiquimula"),
    ("Quetzaltepeque", "Chiquimula"),
    ("Villa Nueva", "Guatemala"),
    ("Pochuta", "Chimaltenango"),
    ("Cuilapa", "Santa Rosa"),
    ("San Bartolo", "Totonicapán"),
    ("Santo Tomás La Unión", "Suchitepéquez"),
    ("Rio Blanco", "San Marcos"),
    ("Barillas", "Huehuetenango"),
    ("Chinique", "Quiché"),
    ("El Chol", "Baja Verapaz"),
    ("San Juan Erminta", "Chiquimula"),
    ("San Jacinto", "Chiquimula"),
    ("San José Pinula", "Guatemala"),
    ("Amatitlán", "Guatemala"),
    ("Patzicía", "Chimaltenango"),
    ("San Juan Cotzal", "Quiché"),
    ("Sacapulas", "Quiché"),
    ("Chajul", "Quiché"),
    ("Melchor de Mencos", "Petén"),
    ("La Unión", "Zacapa"),
    ("Soloma", "Huehuetenango"),
    ("Cunén", "Quiché"),
    ("San José Acatempa", "Jutiapa"),
    ("Antigua Guatemala", "Sacatepéquez"),
    ("Chimaltenango", "Chimaltenango"),
    ("Cabricán", "Quetzaltenango"),
    ("San Mateo Ixtatán", "Huehuetenango"),
    ("Río Hondo", "Zacapa"),
    ("Jerez", "Jutiapa"),
    ("San Miguel Dueñas", "Sacatepéquez"),
    ("Santa Cruz Balanyá", "Chimaltenango"),
    ("Escuintla", "Escuintla"),
    ("Quetzaltenango", "Quetzaltenango"),
    ("Nueva Concepción", "Escuintla"),
    ("Nentón", "Huehuetenango"),
    ("Panzós", "Alta Verapaz"),
    ("Santa Cruz del Quiché", "Quiché"),
    ("San Francisco El Alto", "Totonicapán"),
    ("San Juan Sacatepéquez", "Guatemala"),
    ("Chiquimula", "Chiquimula"),
    ("Nuevo Progreso", "San Marcos"),
    ("Huehuetenango", "Huehuetenango"),
    ("Chiché", "Quiché"),
    ("San Juan Ixcoy", "Huehuetenango"),
    ("Santa Eulalia", "Huehuetenango"),
    ("San Antonio Palopó", "Sololá"),
    ("Senahú", "Alta Verapaz"),
    ("Totonicapán", "Totonicapán"),
    ("La Esperanza", "Quetzaltenango"),
    ("Mazatenango", "Suchitepéquez"),
    ("San Rafael La Independencia", "Huehuetenango"),
    ("Chicamán", "Quiché"),
    ("El Quetzal", "San Marcos"),
    ("San Martín Sacatepéquez", "Quetzaltenango"),
    ("Ignorado", "Desconocido"),
    ("El Tumbador", "San Marcos"),
    ("Santo Domingo Suchitepéquez", "Suchitepéquez"),
    ("Chahal", "Alta Verapaz"),
    ("Zacualpa", "Quiché"),
    ("Fray Bartolomé de Las Casas", "Alta Verapaz"),
    ("Oratorio", "Santa Rosa"),
    ("Villa Canales", "Guatemala"),
    ("Retalhuleu", "Retalhuleu"),
    ("Almolonga", "Quetzaltenango"),
    ("Guastatoya", "El Progreso"),
    ("Chuarrancho", "Guatemala"),
    ("Atescatempa", "Jutiapa"),
    ("Chinautla", "Guatemala"),
    ("San Juan Chamelco", "Alta Verapaz"),
    ("Tamahú", "Alta Verapaz"),
    ("San Sebastián Huehuetenango", "Huehuetenango"),
    ("Cajolá", "Quetzaltenango"),
    ("Flores", "Petén"),
    ("San Pablo", "San Marcos"),
    ("Tecpán Guatemala", "Chimaltenango"),
    ("Iztapa", "Escuintla"),
    ("La Libertad", "Petén"),
    ("Tiquisate", "Escuintla"),
    ("Esquipulas", "Chiquimula"),
    ("Pachalum", "Quiché"),
    ("Malacatán", "San Marcos"),
    ("San Lorenzo", "San Marcos"),
    ("San Miguel Siguilá", "Quetzaltenango"),
    ("Nahualá", "Sololá"),
    ("Santa María de Jesús", "Sacatepéquez"),
    ("Camotán", "Chiquimula"),
    ("Olopa", "Chiquimula"),
    ("San Vicente Pacaya", "Escuintla"),
    ("San José La Arada", "Chiquimula"),
    ("Pajapita", "San Marcos"),
    ("San Pedro Jocopilas", "Quiché"),
    ("Fraijanes", "Guatemala"),
    ("Palencia", "Guatemala"),
    ("Santa Catarina Pinula", "Guatemala"),
    ("Magdalena Milpas Altas", "Sacatepéquez"),
    ("Siquinalá", "Escuintla"),
    ("Santa Clara La Laguna", "Sololá"),
    ("El Tejar", "Chimaltenango"),
    ("San José Poaquil", "Chimaltenango"),
    ("Sibilia", "Quetzaltenango"),
    ("Casillas", "Santa Rosa"),
    ("Panajachel", "Sololá"),
    ("San Andrés Itzapa", "Chimaltenango"),
    ("Salcajá", "Quetzaltenango"),
    ("Coatepeque", "Quetzaltenango"),
    ("Taxisco", "Santa Rosa"),
    ("El Palmar", "Quetzaltenango"),
    ("San Francisco Zapotitlán", "Suchitepéquez"),
    ("Chiquimulilla", "Santa Rosa"),
    ("Jalpatagua", "Jutiapa"),
    ("San Bernardino", "Suchitepéquez"),
    ("Santa Catarina Ixtahuacán", "Sololá"),
    ("Santa Lucía Cotzumalguapa", "Escuintla"),
    ("San Pedro Sacatepéquez", "Guatemala"),
    ("San Marcos", "San Marcos"),
    ("Todos Santos Cuchumatán", "Huehuetenango"),
    ("San Pedro Necta", "Huehuetenango"),
    ("Guanagazapa", "Escuintla"),
    ("Colotenango", "Huehuetenango"),
    ("Canillá", "Quiché"),
    ("Sipacapa", "San Marcos"),
    ("Santa Ana Huista", "Huehuetenango"),
    ("Joyabaj", "Quiché"),
    ("San Antonio Ilotenango", "Quiché"),
    ("Sibinal", "San Marcos"),
    ("San Andrés Sajcabajá", "Quiché"),
    ("San Pedro Carchá", "Alta Verapaz"),
    ("Tactic", "Alta Verapaz"),
    ("Cobán", "Alta Verapaz"),
    ("San Cristóbal Verapaz", "Alta Verapaz"),
    ("Chisec", "Alta Verapaz"),
    ("San Antonio Suchitepéquez", "Suchitepéquez"),
    ("San Andrés Villa Seca", "Suchitepéquez"),
    ("Sayaxché", "Petén"),
    ("El Estor", "Izabal"),
    ("Zunilito", "Suchitepéquez"),
    ("Concepción Chiquirichapa", "Quetzaltenango"),
    ("Cuyotenango", "Suchitepéquez"),
    ("Zaragoza", "Chimaltenango"),
    ("Santa Lucía Milpas Altas", "Sacatepéquez"),
    ("San Miguel Acatán", "Huehuetenango"),
    ("San Agustín Acasaguastlán", "El Progreso"),
    ("Jacaltenango", "Huehuetenango"),
    ("Raxruhá", "Alta Verapaz"),
    ("Santa Rosa de Lima", "Santa Rosa"),
    ("Chicacao", "Suchitepéquez"),
    ("Livingston", "Izabal"),
    ("Nueva Santa Rosa", "Santa Rosa"),
    ("Purulhá", "Baja Verapaz"),
    ("Sumpango", "Sacatepéquez"),
    ("San Luis Jilotepeque", "Jalapa"),
    ("San Juan La Laguna", "Sololá"),
    ("Pueblo Nuevo", "Suchitepéquez"),
    ("Tejutla", "San Marcos"),
    ("Las cruces", "Petén"),
    ("San Gabriel", "Suchitepéquez"),
    ("La Democracia", "Huehuetenango"),
    ("El Progreso", "Jutiapa"),
    ("Santiago Chimaltenango", "Huehuetenango"),
    ("San Francisco", "Petén"),
    ("San Andrés", "Petén"),
    ("Quesada", "Jutiapa"),
    ("San Pedro Ayampuc", "Guatemala"),
    ("Parramos", "Chimaltenango"),
    ("Barberena", "Santa Rosa"),
    ("San Martín Jilotepeque", "Chimaltenango"),
    ("San Andrés Xecul", "Totonicapán"),
    ("Catarina", "San Marcos"),
    ("Champerico", "Retalhuleu"),
    ("San Antonio Aguas Calientes", "Sacatepéquez"),
    ("Zacapa", "Zacapa"),
    ("Santiago Atitlán", "Sololá"),
    ("Morales", "Izabal"),
    ("Granados", "Baja Verapaz"),
    ("Asunción Mita", "Jutiapa"),
    ("La Blanca", "San Marcos"),
    ("Mataquescuintla", "Jalapa"),
    ("Cantel", "Quetzaltenango"),
    ("San José Ojetenam", "San Marcos"),
    ("San Felipe", "Retalhuleu"),
    ("Esquipulas Palo Gordo", "San Marcos"),
    ("Santa Cruz Verapaz", "Alta Verapaz"),
    ("Santa Catarina Barahona", "Sacatepéquez"),
    ("La Gomera", "Escuintla"),
    ("Santa Bárbara", "Huehuetenango"),
    ("San Gaspar Ixchil", "Huehuetenango"),
    ("Cuilco", "Huehuetenango"),
    ("San José El Ídolo", "Suchitepéquez"),
    ("Pueblo Nuevo", "San Marcos"),
    ("San Rafael Pie de La Cuesta", "San Marcos"),
    ("Patzún", "Chimaltenango"),
    ("Concepción Huista", "Huehuetenango"),
    ("Usumatlán", "Zacapa"),
    ("Santa Catarina Palopó", "Sololá"),
    ("Huitán", "Quetzaltenango"),
    ("Palestina de Los Altos", "Quetzaltenango"),
    ("Salamá", "Baja Verapaz"),
    ("Ayutla", "San Marcos"),
    ("Acatenango", "Chimaltenango"),
    ("Santa María Chiquimula", "Totonicapán"),
    ("Chiantla", "Huehuetenango"),
    ("San Juan Bautista", "Suchitepéquez"),
    ("Ciudad Vieja", "Sacatepéquez"),
    ("GUATEMALA", "Guatemala"),
    ("CHINAUTLA", "Guatemala"),
    ("SAN PEDRO AYAMPUC", "Guatemala"),
    ("FRAIJANES", "Guatemala"),
    ("SAN JOSÉ PINULAC", "Guatemala"),
    ("MIXCO", "Guatemala"),
    ("VILLA NUEVA", "Guatemala"),
    ("VILLA CANALES", "Guatemala"),
    ("SANTA CATARINA PINULA", "Guatemala"),
    ("SAN JOSÉ DEL GOLFO", "Guatemala"),
    ("PALENCIA", "Guatemala"),
    ("SAN JUAN SACATEPÉQUEZ", "Guatemala"),
    ("SAN RAIMUNDO", "Guatemala"),
    ("AMATITLÁN", "Guatemala"),
    ("PETAPA", "Guatemala"),
    ("ESCUINTLA", "Escuintla"),
    ("SAN PEDRO SACATEPÉQUEZ", "Guatemala"),
    ("JUTIAPA", "Jutiapa"),
    ("SANTA CRUZ DEL QUICHÉ", "Quiché"),
    ("SAN MIGUEL IXTAHUACÁN", "Huehuetenango"),
    ("SAN PEDRO PINULA", "Jalapa"),
    ("SANTA LUCÍA COTZUMALGUAPA", "Escuintla"),
    ("CHUARRANCHO", "Guatemala"),
    ("PALÍN", "Escuintla"),
    ("ANTIGUA GUATEMALA", "Sacatepéquez"),
    ("SAN JOSÉ", "Escuintla"),
    ("BARBERENA", "Santa Rosa"),
    ("JALAPA", "Jalapa"),
    ("IGNORADO", "Desconocido"),
    ("SAN LUIS JILITEPEQUE", "Jalapa"),
    ("TECPÁN GUATEMALA", "Chimaltenango"),
    ("GUASTATOYA", "El Progreso"),
    ("SAN LUCAS SACATEPÉQUEZ", "Sacatepéquez"),
    ("CABRICÁN", "Quetzaltenango"),
    ("NUEVA SANTA ROSA", "Santa Rosa"),
    ("MAZATENANGO", "Suchitepéquez"),
    ("PUERTO BARRIOS", "Izabal"),
    ("MORAZÁN", "El Progreso"),
    ("LIVINGSTON", "Izabal"),
    ("ZARAGOZA", "Chimaltenango"),
    ("CHIMALTENANGO", "Chimaltenango"),
    ("SAN JOSÉ POAQUIL", "Chimaltenango"),
    ("SAN MARTÍN JILOTEPEQUE", "Chimaltenango"),
    ("COMALAPA", "Chimaltenango"),
    ("PATZICÍA", "Chimaltenango"),
    ("SANTA APOLONIA", "Chimaltenango"),
    ("PATZÚN", "Chimaltenango"),
    ("POCHUTA", "Chimaltenango"),
    ("ACATENANGO", "Chimaltenango"),
    ("YEPOCAPA", "Chimaltenango"),
    ("SAN ANDRÉS ITZAPA", "Chimaltenango"),
    ("PARRAMOS", "Chimaltenango"),
    ("EL TEJAR", "Chimaltenango"),
    ("SACAPULAS", "Quiché"),
    ("SANTA CRUZ BALANYÁ", "Chimaltenango"),
    ("SANTA MARÍA DE JESÚS", "Sacatepéquez"),
    ("SAN FRANCISCO EL ALTO", "Totonicapán"),
    ("SUMPANGO", "Sacatepéquez"),
    ("LA DEMOCRACIA", "Escuintla"),
    ("SIQUINALÁ", "Escuintla"),
    ("MASAGUA", "Escuintla"),
    ("TIQUISATE", "Escuintla"),
    ("LA GOMERA", "Escuintla"),
    ("GUANAGAZAPA", "Escuintla"),
    ("IZTAPA", "Escuintla"),
    ("SAN VICENTE PACAYA", "Escuintla"),
    ("NUEVA CONCEPCIÓN", "Escuintla"),
    ("NUEVO PROGRESO", "San Marcos"),
    ("SANTA ROSA DE LIMA", "Santa Rosa"),
    ("CUILAPA", "Santa Rosa"),
    ("GUAZACAPÁN", "Santa Rosa"),
    ("ORATORIO", "Santa Rosa"),
    ("PUEBLO NUEVO VIÑAS", "Santa Rosa"),
    ("SANTA MARÍA IXHUATÁN", "Santa Rosa"),
    ("SANTA CRUZ NARANJO", "Santa Rosa"),
    ("CASILLAS", "Santa Rosa"),
    ("SAN RAFAEL LAS FLORES", "Santa Rosa"),
    ("CHIQUIMULILLA", "Santa Rosa"),
    ("TAXISCO", "Santa Rosa"),
    ("SAN JUAN TECUACO", "Santa Rosa"),
    ("MATAQUESCUINTLA", "Jalapa"),
    ("SAN JOSÉ CHACAYÁ", "Sololá"),
    ("NAHUALÁ", "Sololá"),
    ("SANTA CATARINA IXTAHUACÁN", "Sololá"),
    ("CONCEPCIÓN", "Sololá"),
    ("SAN ANDRÉS SEMETABAJ", "Sololá"),
    ("SANTA CATARINA PALOPÓ", "Sololá"),
    ("SANTA CRUZ LA LAGUNA", "Sololá"),
    ("SAN JUAN LA LAGUNA", "Sololá"),
    ("SANTIAGO ATITLÁN", "Sololá"),
    ("SANTA CLARA LA LAGUNA", "Sololá"),
    ("SAN LUCAS TOLIMÁN", "Sololá"),
    ("SOLOLÁ", "Sololá"),
    ("SANTA LUCÍA UTATLÁN", "Sololá"),
    ("PANAJACHEL", "Sololá"),
    ("SAN ANTONIO PALOPÓ", "Sololá"),
    ("SAN PEDRO LA LAGUNA", "Sololá"),
    ("SAN PABLO LA LAGUNA", "Sololá"),
    ("SAN MARCOS LA LAGUNA", "Sololá"),
    ("TOTONICAPÁN", "Totonicapán"),
    ("SAN ANDRÉS XECUL", "Totonicapán"),
    ("SAN CRISTÓBAL TOTONICAPÁN", "Totonicapán"),
    ("MONOSTENANGO", "Totonicapán"),
    ("SAN BARTOLO", "Totonicapán"),
    ("SAN LUCÍA LA REFORMA", "Totonicapán"),
    ("SANTA MARÍA CHIQUIMULA", "Totonicapán"),
    ("QUETZALTENANGO", "Quetzaltenango"),
    ("SAN FRANCISCO ZAPOTITLÁN", "Quetzaltenango"),
    ("CAJOLÁ", "Quetzaltenango"),
    ("SAN CARLOS SIJA", "Quetzaltenango"),
    ("SAN MATEO", "Quetzaltenango"),
    ("SALCAJÁ", "Quetzaltenango"),
    ("OLINTEPEQUE", "Quetzaltenango"),
    ("SIBILIA", "Quetzaltenango"),
    ("OSTUNCALCO", "Quetzaltenango"),
    ("CONCEPCIÓN CHIQUIRICHAPA", "Quetzaltenango"),
    ("SAN MARTÍN SACATEPÉQUEZ", "Quetzaltenango"),
    ("ALMOLONGA", "Quetzaltenango"),
    ("CANTEL", "Quetzaltenango"),
    ("ZUNIL", "Quetzaltenango"),
    ("HUITÁN", "Quetzaltenango"),
    ("COLOMBA", "Quetzaltenango"),
    ("EL PALMAR", "Quetzaltenango"),
    ("COATEPEQUE", "Quetzaltenango"),
    ("GÉNOVA", "Quetzaltenango"),
    ("FLORES COSTA CUCA", "Quetzaltenango"),
    ("PAJAPITA", "San Marcos"),
    ("LA ESPERANZA", "Quetzaltenango"),
    ("EL QUETZAL", "San Marcos"),
    ("PALESTINA DE LOS ALTOS", "Quetzaltenango"),
    ("SAN MIGUEL SIGÜILÁ", "Quetzaltenango"),
    ("SAN ANTONIO SACATEPÉQUEZ", "Sacatepéquez"),
    ("SAN JUAN COTZAL", "Quiché"),
    ("SAN FRANCISCO LA UNIÓN", "Quiché"),
    ("CUYOTENANGO", "Suchitepéquez"),
    ("SAN SEBASTIÁN", "Quetzaltenango"),
    ("CHAPERICO", "Retalhuleu"),
    ("RETALHULEU", "Retalhuleu"),
    ("NUEVO SAN CARLOS", "Retalhuleu"),
    ("SAN ANDRÉS VILLA SECA", "Suchitepéquez"),
    ("SAN MARTÍN ZAPOTITLÁN", "Retalhuleu"),
    ("SAN FELIPE", "Retalhuleu"),
    ("EL ASINTAL", "Retalhuleu"),
    ("SANTA CRUZ MULUÁ", "Retalhuleu"),
    ("HUEHUETENANGO", "Huehuetenango"),
    ("SAN MARCOS", "San Marcos"),
    ("SAN RAFAEL PIÉ DE LA CUESTA", "San Marcos"),
    ("COMITANCILLO", "San Marcos"),
    ("CONCEPCIÓN TUTUAPA", "San Marcos"),
    ("TACANÁ", "San Marcos"),
    ("SIBINAL", "San Marcos"),
    ("TAJUMULCO", "San Marcos"),
    ("TEJUTLA", "San Marcos"),
    ("EL TUMBADOR", "San Marcos"),
    ("EL RODEO", "San Marcos"),
    ("MALACATÁN", "San Marcos"),
    ("OCÓS", "San Marcos"),
    ("SAN PABLO", "San Marcos"),
    ("LA REFORMA", "San Marcos"),
    ("IXCHIGUÁN", "San Marcos"),
    ("SIPACAPA", "San Marcos"),
    ("ESQUIPULAS PALO GORDO", "San Marcos"),
    ("RÍO BLANCO", "San Marcos"),
    ("SAN LORENZO", "San Marcos"),
    ("LA BLANCA", "San Marcos"),
    ("CATARINA", "San Marcos"),
    ("SAN JOSÉ OJETENAM", "San Marcos"),
    ("SAN CRISTÓBAL CUCHO", "San Marcos"),
    ("AYUTLA", "San Marcos"),
    ("CHIANTLA", "Huehuetenango"),
    ("SANTA BÁRBARA", "Huehuetenango"),
    ("AGUACATÁN", "Huehuetenango"),
    ("LA DEMOCRACIA", "Huehuetenango"),
    ("COLOTENANGO", "Huehuetenango"),
    ("SAN SEBASTIÁN HUEHUETENANGO", "Huehuetenango"),
    ("TECTITÁN", "Huehuetenango"),
    ("MALACATANCITO", "Huehuetenango"),
    ("CUILCO", "Huehuetenango"),
    ("SAN PEDRO NECTA", "Huehuetenango"),
    ("SOLOMA", "Huehuetenango"),
    ("LA LIBERTAD", "Huehuetenango"),
    ("SAN MIGUEL ACATÁN", "Huehuetenango"),
    ("SAN RAFAEL LA INDEPENDENCIA", "Huehuetenango"),
    ("TODOS SANTOS CUCHUMATÁN", "Huehuetenango"),
    ("BARILLAS", "Huehuetenango"),
    ("SANTA EULALIA", "Huehuetenango"),
    ("SAN JUAN IXCOY", "Huehuetenango"),
    ("SAN ANTONIO HUISTA", "Huehuetenango"),
    ("SAN SEBASTIÁN COATÁN", "Huehuetenango"),
    ("SAN GASPAR IXCHIL", "Huehuetenango"),
    ("SAN JUAN ATITÁN", "Huehuetenango"),
    ("NENTÓN", "Huehuetenango"),
    ("JACALTENANGO", "Huehuetenango"),
    ("IXTAHUCÁN", "Huehuetenango"),
    ("SAN RAFAEL PETZAL", "Huehuetenango"),
    ("PETATAN", "Huehuetenango"),
    ("SANTIAGO CHIMALTENANGO", "Huehuetenango"),
    ("SANTA ANA HUISTA", "Huehuetenango"),
    ("CONCEPCIÓN HUISTA", "Huehuetenango"),
    ("IXCÁN", "Quiché"),
    ("ZACUALPA", "Quiché"),
    ("CHAJUL", "Quiché"),
    ("CHICHICASTENANGO", "Quiché"),
    ("PATZITÉ", "Quiché"),
    ("SAN ANTONIO ILOTENANGO", "Quiché"),
    ("CUNÉN", "Quiché"),
    ("JOYABAJ", "Quiché"),
    ("NEBAJ", "Quiché"),
    ("SAN ANDRÉS SAJCABAJÁ", "Quiché"),
    ("SAN BARTALOMÉ JOCOTENANGO", "Quiché"),
    ("CANILLÁ", "Quiché"),
    ("CHICAMÁN", "Quiché"),
    ("PACHALUM", "Quiché"),
    ("CHICHÉ", "Quiché"),
    ("CHINIQUE", "Quiché"),
    ("COBÁN", "Alta Verapaz"),
    ("USPANTÁN", "Quiché"),
    ("SALAMÁ", "Baja Verapaz"),
    ("SAN MIGUEL CHICAJ", "Baja Verapaz"),
    ("RABINAL", "Baja Verapaz"),
    ("CUBULCO", "Baja Verapaz"),
    ("GRANADOS", "Baja Verapaz"),
    ("SAN JERÓNIMO", "Baja Verapaz"),
    ("PURULHÁ", "Baja Verapaz"),
    ("EL CHOL", "Baja Verapaz"),
    ("SAN PEDRO CARCHÁ", "Alta Verapaz"),
    ("SAN JUAN CHAMELCO", "Alta Verapaz"),
    ("SANTA CRUZ VERAPAZ", "Alta Verapaz"),
    ("SAN CRISTÓBAL VERAPAZ", "Alta Verapaz"),
    ("TACTIC", "Alta Verapaz"),
    ("TAMAHÚ", "Alta Verapaz"),
    ("TUCURÚ", "Alta Verapaz"),
    ("PANZÓS", "Alta Verapaz"),
    ("SENAHÚ", "Alta Verapaz"),
    ("LANQUÍN", "Alta Verapaz"),
    ("CHISEC", "Alta Verapaz"),
    ("CHAHAL", "Alta Verapaz"),
    ("FRAY BARTOLOMÉ DE LAS CASAS", "Alta Verapaz"),
    ("SAN CATALINA LA TINTA", "Alta Verapaz"),
    ("RAXRUJÀ", "Alta Verapaz"),
    ("CAHABÓN", "Alta Verapaz"),
    ("SAYAXCHÉ", "Petén"),
    ("SAN LUIS", "Petén"),
    ("FLORES", "Petén"),
    ("SAN BENITO", "Petén"),
    ("SAN ANDRÉS", "Petén"),
    ("SAN FRANCISCO", "Petén"),
    ("DOLORES", "Petén"),
    ("EL CHAL", "Petén"),
    ("MELCHOR DE MENCOS", "Petén"),
    ("POPTÚN", "Petén"),
    ("LAS CRUCES", "Petén"),
    ("SANTA ANA", "Petén"),
    ("GUALÁN", "Zacapa"),
    ("ASUNCIÓN MITA", "Jutiapa"),
    ("SAN ANTONIO LA PAZ", "El Progreso"),
    ("EL JÍCARO", "El Progreso"),
    ("SAN AGUSTÍN ACASAGUASTLAN", "El Progreso"),
    ("SANARATE", "El Progreso"),
    ("SAN CRISTÓBAL ACASAGUASTLAN", "El Progreso"),
    ("SANSARE", "El Progreso"),
    ("AGUA BLANCA", "Jutiapa"),
    ("JOCOTENANGO", "Sacatepéquez"),
    ("PASTORES", "Sacatepéquez"),
    ("SANTO DOMINGO XENACOJ", "Sacatepéquez"),
    ("SANTIAGO SACATEPÉQUEZ", "Sacatepéquez"),
    ("SAN BARTOLOMÉ MILPAS ALTAS", "Sacatepéquez"),
    ("SANTA LUCÍA MILPAS ALTAS", "Sacatepéquez"),
    ("MAGDALENA MILPAS ALTAS", "Sacatepéquez"),
    ("CIUDAD VIEJA", "Sacatepéquez"),
    ("SAN MIGUEL DUEÑAS", "Sacatepéquez"),
    ("ALOTENANGO", "Sacatepéquez"),
    ("SANTA CATARINA BARAHONA", "Sacatepéquez"),
    ("SAMAYAC", "Suchitepéquez"),
    ("SANTO DOMINGO SUCHITEPÉQUEZ", "Suchitepéquez"),
    ("SAN ANTONIO SUCHITEPÉQUEZ", "Suchitepéquez"),
    ("ZUNILITO", "Suchitepéquez"),
    ("SAN JOSÉ LA MÁQUINA", "Suchitepéquez"),
    ("SAN BERNARDINO", "Suchitepéquez"),
    ("SAN JOSÉ EL ÍDOLO", "Suchitepéquez"),
    ("SAN PABLO JOCOPILAS", "Suchitepéquez"),
    ("SAN GABRIEL", "Suchitepéquez"),
    ("CHICACAO", "Suchitepéquez"),
    ("SAN JUAN BAUTISTA", "Suchitepéquez"),
    ("PATULUL", "Suchitepéquez"),
    ("SANTO TOMÁS LA UNIÓN", "Suchitepéquez"),
    ("PUEBLO NUEVO", "Suchitepéquez"),
    ("RÍO BRAVO", "Suchitepéquez"),
    ("MORALES", "Izabal"),
    ("EL ESTOR", "Izabal"),
    ("ZACAPA", "Zacapa"),
    ("TECULUTÁN", "Zacapa"),
    ("SAN JORGE", "Zacapa"),
    ("ESTANZUELA", "Zacapa"),
    ("RÍO HONDO", "Zacapa"),
    ("HUITÉ", "Zacapa"),
    ("ESQUIPULAS", "Chiquimula"),
    ("SAN JACINTO", "Chiquimula"),
    ("CHIQUIMULA", "Chiquimula"),
    ("OLOPA", "Chiquimula"),
    ("SAN JUAN ERMITA", "Chiquimula"),
    ("JOCOTÁN", "Chiquimula"),
    ("CONCEPCIÓN LAS MINAS", "Chiquimula"),
    ("QUETZALTEPEQUE", "Chiquimula"),
    ("IPALA", "Chiquimula"),
    ("MONJAS", "Jalapa"),
    ("SAN MANUEL CHAPARRÓN", "Jalapa"),
    ("CONGUACO", "Jutiapa"),
    ("SANTA CATARINA MITA", "Jutiapa"),
    ("YUPILTEPEQUE", "Jutiapa"),
    ("COMAPA", "Jutiapa"),
    ("JALPATAGUA", "Jutiapa"),
    ("EL PROGRESO", "Jutiapa"),
    ("ATESCATEMPA", "Jutiapa"),
    ("JEREZ", "Jutiapa"),
    ("ZAPOTITLÁN", "Jutiapa"),
    ("PASACO", "Jutiapa"),
    ("QUEZADA", "Jutiapa"),
    ("SAN MIGUEL PANÁN", "Jutiapa"),
    ("LOS AMATES", "Izabal"),
    ("USUMATLÁN", "Zacapa"),
    ("SAN DIEGO", "Zacapa"),
    ("LA UNIÓN", "Zacapa"),
    ("SAN JOSÉ LA ARADA", "Chiquimula"),
    ("EL ADELANTO", "Jutiapa"),
    ("SAN JOSÉ ACATEMPA", "Jutiapa"),
    ("SAN ANTONIO AGUAS CALIENTES", "Sacatepéquez"),
    ("CABAÑAS", "Zacapa"),
    ("CAMOTÁN", "Chiquimula"),
    ("MOYUTA", "Jutiapa"),
    ("SAN MATEO IXTATÁN", "Huehuetenango"),
    ("La Tinta", "Alta Verapaz"),
    ("Fray Bartolomé de las Casas", "Alta Verapaz"),
    ("Guanazapa", "Escuintla"),
    ("San Joseé La Máquina", "Suchitepéquez"),
    ("San Cristobal Acasaguastlán", "El Progreso"),
    ("Zapotitlán", "Jutiapa"),
    ("San Francisco el Alto", "Totonicapán"),
    ("Santa Lucia Utatlán", "Sololá"),
    ("San Antonio la Paz", "El Progreso"),
    ("San Francisco la Unión", "Quiché"),
    ("Santiago Sactepéquez", "Sacatepéquez"),
    ("El Jícaro", "El Progreso"),
    ("Santa Catalina Ixtahuacán", "Sololá"),
    ("Santo Tomás la Unión", "Suchitepéquez"),
    ("San Andres Semetabaj", "Sololá"),
    ("Melchor", "Petén"),
    ("San Rafael la Independencia", "Huehuetenango"),
    ("Santiago Chimaltenanango", "Huehuetenango"),
    ("Poptún", "Petén"),
    ("San Luis", "Petén"),
    ("Comitancillo", "San Marcos"),
    ("Dolores", "Petén"),
    ("San Antonio Sacatepéquez", "San Marcos"),
    ("Nuevo San Carlos", "Retalhuleu"),
    ("San Miguel Ixtahuacán", "San Marcos"),
    ("San Lucas Sacatepéquez", "Sacatepéquez"),
    ("Rabinal", "Baja Verapaz"),
    ("San Miguel Chicaj", "Baja Verapaz"),
    ("San José el Golfo", "Guatemala"),
    ("San Raimundo", "Guatemala"),
    ("Petapa", "Guatemala"),
    ("Guatemala", "Guatemala"),
    ("Alotenango", "Sacatepéquez"),
    ("Chimaltenango", "Chimaltenango"),
    ("Yepocapa", "Chimaltenango"),
    ("San Rafael las Flores", "Santa Rosa"),
    ("Santiago Atitlán", "Sololá"),
    ("San Cristobal Totonicapán", "Totonicapán"),
    ("Santa Lucía La Reforma", "Totonicapán"),
    ("Salcajá", "Quetzaltenango"),
    ("Olintepeque", "Quetzaltenango"),
    ("Ostuncalco", "Quetzaltenango"),
    ("Colomba", "Quetzaltenango"),
    ("Palestina de los Altos", "Quetzaltenango"),
    ("San José El Idolo", "Suchitepéquez"),
    ("Patulul", "Suchitepéquez"),
    ("Santo Tomás La Unión", "Suchitepéquez"),
    ("Rio Bravo", "Escuintla"),
    ("San Rafael Pié de La Cuesta", "San Marcos"),
    ("San Marcos", "San Marcos"),
    ("El Rodeo", "San Marcos"),
    ("Ocós", "San Marcos"),
    ("Ixtahuacán", "Huehuetenango"),
    ("San Antonio Huista", "Huehuetenango"),
    ("Chichicastenango", "Quiché"),
    ("Uspantán", "Quiché"),
    ("Ixcán", "Quiché"),
    ("Nebaj", "Quiché"),
    ("Tucurú", "Alta Verapaz"),
    ("Lanquín", "Alta Verapaz"),
    ("Cahabón", "Alta Verapaz"),
    ("Santa Catalina la Tinta", "Alta Verapaz"),
    ("Jocotán", "Chiquimula"),
    ("Quetzaltepeque", "Chiquimula"),
    ("San José Acatempa", "Jutiapa"),
    ("Villa Nueva", "Guatemala"),
    ("Comalapa", "Chimaltenango"),
    ("Pochuta", "Chimaltenango"),
    ("Santa Lucía Cotzumalguapa", "Escuintla"),
    ("San Bartolo", "Totonicapán"),
    ("Quetzaltenango", "Quetzaltenango"),
    ("Mazatenango", "Suchitepéquez"),
    ("El Asintal", "Retalhuleu"),
    ("Rio Blanco", "San Marcos"),
    ("Barillas", "Huehuetenango"),
    ("Santa Cruz del Quiché", "Quiché"),
    ("El Chol", "Baja Verapaz"),
    ("Chiquimula", "Chiquimula"),
    ("San Juan Erminta", "Chiquimula"),
    ("San Agustín Acasaguastlán", "El Progreso"),
    ("Totonicapán", "Totonicapán"),
    ("San Francisco La Unión", "Quiché"),
    ("Coatepeque", "Quetzaltenango"),
    ("Unión Cantinil", "Huehuetenango"),
    ("Chinique", "Quiché"),
    ("Antigua Guatemala", "Sacatepéquez"),
    ("Amatitlán", "Guatemala"),
    ("San Antonio Sacatepéquez", "San Marcos"),
    ("Huehuetenango", "Huehuetenango"),
    ("Cobán", "Alta Verapaz"),
    ("Jalapa", "Jalapa"),
    ("Nueva Concepción", "Escuintla"),
    ("San Francisco El Alto", "Totonicapán"),
    ("Soloma", "Huehuetenango"),
    ("Río Hondo", "Zacapa"),
    ("Jutiapa", "Jutiapa"),
    ("Jerez", "Jutiapa"),
    ("Malacatán", "San Marcos"),
    ("San Lucas Toliman", "Sololá"),
    ("San Mateo Ixtatán", "Huehuetenango"),
    ("Tamahú", "Alta Verapaz"),
    ("Flores", "Petén"),
    ("Camotán", "Chiquimula"),
    ("San José Chacayá", "Sololá"),
    ("Cabricán", "Quetzaltenango"),
    ("El Tumbador", "San Marcos"),
    ("San Juan Sacatepéquez", "Guatemala"),
    ("Villa Canales", "Guatemala"),
    ("Cajolá", "Quetzaltenango"),
    ("La Esperanza", "Quetzaltenango"),
    ("El Quetzal", "San Marcos"),
    ("Sibinal", "San Marcos"),
    ("Patzún", "Chimaltenango"),
    ("Parramos", "Chimaltenango"),
    ("Flores Costa Cuca", "Quetzaltenango"),
    ("Cuilapa", "Santa Rosa"),
    ("Colotenango", "Huehuetenango"),
    ("Chajul", "Quiché"),
    ("El Estor", "Izabal"),
    ("Mixco", "Guatemala"),
    ("San Andrés Itzapa", "Chimaltenango"),
    ("Palín", "Escuintla"),
    ("Casillas", "Santa Rosa"),
    ("Pajapita", "San Marcos"),
    ("San Andrés Xecul", "Totonicapán"),
    ("La Democracia", "Escuintla"),
    ("Sololá", "Sololá"),
    ("Esquipulas", "Chiquimula"),
    ("San Carlos Sija", "Quetzaltenango"),
    ("Chicamán", "Quiché"),
    ("Ayutla", "San Marcos"),
    ("Salamá", "Baja Verapaz"),
    ("Concepción Chiquirichapa", "Quetzaltenango"),
    ("Tecpán Guatemala", "Chimaltenango"),
    ("San Antonio Ilotenango", "Quiché"),
    ("Momostenango", "Totonicapán"),
    ("San Pedro Necta", "Huehuetenango"),
    ("Pastores", "Sacatepéquez"),
    ("Cantel", "Quetzaltenango"),
    ("Chiché", "Quiché"),
    ("Siquinalá", "Escuintla"),
    ("Chuarrancho", "Guatemala"),
    ("San Bernardino", "Suchitepéquez"),
    ("Rabinal", "Baja Verapaz"),
    ("San Martín Zapotitlán", "Retalhuleu"),
    ("Santa Catarina Pinula", "Guatemala"),
    ("Chicacao", "Suchitepéquez"),
    ("San Cristóbal Verapaz", "Alta Verapaz"),
    ("San José Poaquil", "Chimaltenango"),
    ("San Vicente Pacaya", "Escuintla"),
    ("San Andrés Sajcabajá", "Quiché"),
    ("San Antonio Suchitepéquez", "Suchitepéquez"),
    ("Taxisco", "Santa Rosa"),
    ("Los Amates", "Izabal"),
    ("Palencia", "Guatemala"),
    ("Fraijanes", "Guatemala"),
    ("San Antonio La Paz", "El Progreso"),
    ("Santo Domingo Xenacoj", "Sacatepéquez"),
    ("Patzicía", "Chimaltenango"),
    ("La Gomera", "Escuintla"),
    ("San José", "Escuintla"),
    ("Santa María Chiquimula", "Totonicapán"),
    ("Sibilia", "Quetzaltenango"),
    ("San Miguel Siguilá", "Quetzaltenango"),
    ("San Francisco Zapotitlán", "Suchitepéquez"),
    ("Santo Domingo Suchitepéquez", "Suchitepéquez"),
    ("Samayac", "Suchitepéquez"),
    ("San Pablo Jocopilas", "Suchitepéquez"),
    ("Santa Bárbara", "Huehuetenango"),
    ("Retalhuleu", "Retalhuleu"),
    ("San Pedro Sacatepéquez", "Guatemala"),
    ("Concepción Tutuapa", "San Marcos"),
    ("Tejutla", "San Marcos"),
    ("Catarina", "San Marcos"),
    ("San Lorenzo", "San Marcos"),
    ("San Miguel Acatán", "Huehuetenango"),
    ("San Juan Atitán", "Huehuetenango"),
    ("Aguacatán", "Huehuetenango"),
    ("San Rafael Petzal", "Huehuetenango"),
    ("Santiago Chimaltenango", "Huehuetenango"),
    ("Santa Ana Huista", "Huehuetenango"),
    ("Zacualpa", "Quiché"),
    ("San Pedro Jocopilas", "Quiché"),
    ("Cunén", "Quiché"),
    ("Joyabaj", "Quiché"),
    ("Sacapulas", "Quiché"),
    ("Canillá", "Quiché"),
    ("Purulhá", "Baja Verapaz"),
    ("Tactic", "Alta Verapaz"),
    ("San Pedro Carchá", "Alta Verapaz"),
    ("Chisec", "Alta Verapaz"),
    ("Chahal", "Alta Verapaz"),
    ("La Unión", "Zacapa"),
    ("Olopa", "Chiquimula"),
    ("San Jacinto", "Chiquimula"),
    ("Asunción Mita", "Jutiapa"),
    ("Chinautla", "Guatemala"),
    ("San Martín Sacatepéquez", "Quetzaltenango"),
    ("Guastatoya", "El Progreso"),
    ("Santa Eulalia", "Huehuetenango"),
    ("Santa Catarina Ixtahuacán", "Sololá"),
    ("San Andrés Semetabaj", "Sololá"),
    ("San Juan Bautista", "Suchitepéquez"),
    ("Pueblo Nuevo", "Suchitepéquez"),
    ("Cuilco", "Huehuetenango"),
    ("San Sebastián Huehuetenango", "Huehuetenango"),
    ("Moyuta", "Jutiapa"),
    ("Sayaxché", "Petén"),
    ("Ipala", "Chiquimula"),
    ("San Juan Cotzal", "Quiché"),
    ("La Libertad", "Petén"),
    ("Las cruces", "Petén"),
    ("Sumpango", "Sacatepéquez"),
    ("Escuintla", "Escuintla"),
    ("San Mateo", "Quetzaltenango"),
    ("Mataquescuintla", "Jalapa"),
    ("San Pedro Ayampuc", "Guatemala"),
    ("Zaragoza", "Chimaltenango"),
    ("Panzós", "Alta Verapaz"),
    ("San Francisco", "Petén"),
    ("San Martín Jilotepeque", "Chimaltenango"),
    ("Pueblo Nuevo Viñas", "Santa Rosa"),
    ("Sipacapa", "San Marcos"),
    ("San Sebastián", "Quetzaltenango"),
    ("Patzité", "Quiché"),
    ("Santa Cruz Verapaz", "Alta Verapaz"),
    ("San Benito", "Petén"),
    ("San Miguel Panán", "Jutiapa"),
    ("San Juan Chamelco", "Alta Verapaz"),
    ("Raxruhá", "Alta Verapaz"),
    ("Almolonga", "Quetzaltenango"),
    ("San Rafael La Independencia", "Huehuetenango"),
    ("Nueva Santa Rosa", "Santa Rosa"),
    ("San Miguel Chicaj", "Baja Verapaz"),
    ("Pasaco", "Jutiapa"),
    ("San Gabriel", "Suchitepéquez"),
    ("Cubulco", "Baja Verapaz"),
    ("Santa Lucía Milpas Altas", "Sacatepéquez"),
    ("Granados", "Baja Verapaz"),
    ("Ciudad Vieja", "Sacatepéquez"),
    ("San Miguel Dueñas", "Sacatepéquez"),
    ("Santiago Sacatepéquez", "Sacatepéquez"),
    ("San Antonio Palopó", "Sololá"),
    ("Santa Cruz Balanyá", "Chimaltenango"),
    ("Poptún", "Petén"),
    ("Santa Cruz Naranjo", "Santa Rosa"),
    ("San José El Ídolo", "Suchitepéquez"),
    ("Pueblo Nuevo", "San Marcos"),
    ("San Rafael Pie de La Cuesta", "San Marcos"),
    ("San Pablo", "San Marcos"),
    ("Usumatlán", "Zacapa"),
    ("Malacatancito", "Huehuetenango"),
    ("Magdalena Milpas Altas", "Sacatepéquez"),
    ("Acatenango", "Chimaltenango"),
    ("Concepción Las Minas", "Chiquimula"),
    ("Yupiltepeque", "Jutiapa"),
    ("Santa Catarina Barahona", "Sacatepéquez"),
    ("Quesada", "Jutiapa"),
    ("El Palmar", "Quetzaltenango"),
    ("El Tejar", "Chimaltenango"),
    ("Fray Bartolomé de Las Casas", "Alta Verapaz"),
    ("Concepción", "Sololá"),
    ("Comitancillo", "San Marcos"),
    ("La Reforma", "San Marcos"),
    ("Nentón", "Huehuetenango"),
    ("San Antonio Aguas Calientes", "Sacatepéquez"),
    ("Atescatempa", "Jutiapa"),
    ("GUATEMALA", "Guatemala"),
    ("VILLA NUEVA", "Guatemala"),
    ("SANTA CATARINA PINULA", "Guatemala"),
    ("SAN JOSÉ PINULAC", "Guatemala"),
    ("SAN JOSÉ DEL GOLFO", "Guatemala"),
    ("PALENCIA", "Guatemala"),
    ("CHINAUTLA", "Guatemala"),
    ("SAN PEDRO AYAMPUC", "Guatemala"),
    ("MIXCO", "Guatemala"),
    ("SAN JUAN SACATEPÉQUEZ", "Guatemala"),
    ("SAN RAIMUNDO", "Guatemala"),
    ("FRAIJANES", "Guatemala"),
    ("AMATITLÁN", "Guatemala"),
    ("VILLA CANALES", "Guatemala"),
    ("PETAPA", "Guatemala"),
    ("CHUARRANCHO", "Guatemala"),
    ("SAN PEDRO SACATEPÉQUEZ", "Guatemala"),
    ("CHIMALTENANGO", "Chimaltenango"),
    ("SAN JOSÉ POAQUIL", "Chimaltenango"),
    ("SAN MARTÍN JILOTEPEQUE", "Chimaltenango"),
    ("COMALAPA", "Chimaltenango"),
    ("SANTA APOLONIA", "Chimaltenango"),
    ("TECPÁN GUATEMALA", "Chimaltenango"),
    ("PATZÚN", "Chimaltenango"),
    ("POCHUTA", "Chimaltenango"),
    ("PATZICÍA", "Chimaltenango"),
    ("ACATENANGO", "Chimaltenango"),
    ("YEPOCAPA", "Chimaltenango"),
    ("SAN ANDRÉS ITZAPA", "Chimaltenango"),
    ("PARRAMOS", "Chimaltenango"),
    ("ZARAGOZA", "Chimaltenango"),
    ("EL TEJAR", "Chimaltenango"),
    ("SANTA CRUZ BALANYÁ", "Chimaltenango"),
    ("SAN MARTÍN SACATEPÉQUEZ", "Quetzaltenango"),
    ("ESCUINTLA", "Escuintla"),
    ("SANTA LUCÍA COTZUMALGUAPA", "Escuintla"),
    ("LA DEMOCRACIA", "Escuintla"),
    ("SIQUINALÁ", "Escuintla"),
    ("MASAGUA", "Escuintla"),
    ("TIQUISATE", "Escuintla"),
    ("LA GOMERA", "Escuintla"),
    ("GUANAGAZAPA", "Escuintla"),
    ("SAN JOSÉ", "Escuintla"),
    ("IZTAPA", "Escuintla"),
    ("PALÍN", "Escuintla"),
    ("SAN VICENTE PACAYA", "Escuintla"),
    ("NUEVA CONCEPCIÓN", "Escuintla"),
    ("CUILAPA", "Santa Rosa"),
    ("BARBERENA", "Santa Rosa"),
    ("SANTA ROSA DE LIMA", "Santa Rosa"),
    ("CASILLAS", "Santa Rosa"),
    ("SAN RAFAEL LAS FLORES", "Santa Rosa"),
    ("ORATORIO", "Santa Rosa"),
    ("CHIQUIMULILLA", "Santa Rosa"),
    ("SANTA MARÍA IXHUATÁN", "Santa Rosa"),
    ("SANTA CRUZ NARANJO", "Santa Rosa"),
    ("PUEBLO NUEVO VIÑAS", "Santa Rosa"),
    ("NUEVA SANTA ROSA", "Santa Rosa"),
    ("TAXISCO", "Santa Rosa"),
    ("GUAZACAPÁN", "Santa Rosa"),
    ("SAN JOSÉ CHACAYÁ", "Sololá"),
    ("NAHUALÁ", "Sololá"),
    ("SANTA CATARINA IXTAHUCÁN", "Sololá"),
    ("CONCEPCIÓN", "Sololá"),
    ("SAN ANDRÉS SEMETABAJ", "Sololá"),
    ("SANTA CATARINA PALOPÓ", "Sololá"),
    ("SANTA CRUZ LA LAGUNA", "Sololá"),
    ("SAN JUAN LA LAGUNA", "Sololá"),
    ("SANTIAGO ATITLÁN", "Sololá"),
    ("SANTA CLARA LA LAGUNA", "Sololá"),
    ("SAN LUCAS TOLIMÁN", "Sololá"),
    ("SOLOLÁ", "Sololá"),
    ("SANTA LUCÍA UTATLÁN", "Sololá"),
    ("PANAJACHEL", "Sololá"),
    ("SAN ANTONIO PALOPÓ", "Sololá"),
    ("SAN PEDRO LA LAGUNA", "Sololá"),
    ("SAN PABLO LA LAGUNA", "Sololá"),
    ("SAN MARCOS LA LAGUNA", "Sololá"),
    ("TOTONICAPÁN", "Totonicapán"),
    ("SAN CRISTÓBAL TOTONICAPÁN", "Totonicapán"),
    ("SAN FRANCISCO EL ALTO", "Totonicapán"),
    ("MONOSTENANGO", "Totonicapán"),
    ("SAN BARTOLO", "Totonicapán"),
    ("SAN ANDRÉS XECUL", "Totonicapán"),
    ("SAN LUCÍA LA REFORMA", "Totonicapán"),
    ("SANTA MARÍA CHIQUIMULA", "Totonicapán"),
    ("QUETZALTENANGO", "Quetzaltenango"),
    ("SALCAJÁ", "Quetzaltenango"),
    ("OLINTEPEQUE", "Quetzaltenango"),
    ("SIBILIA", "Quetzaltenango"),
    ("CABRICÁN", "Quetzaltenango"),
    ("OSTUNCALCO", "Quetzaltenango"),
    ("SAN MATEO", "Quetzaltenango"),
    ("CONCEPCIÓN CHIQUIRICHAPA", "Quetzaltenango"),
    ("ALMOLONGA", "Quetzaltenango"),
    ("CANTEL", "Quetzaltenango"),
    ("HUITÁN", "Quetzaltenango"),
    ("COLOMBA", "Quetzaltenango"),
    ("EL PALMAR", "Quetzaltenango"),
    ("COATEPEQUE", "Quetzaltenango"),
    ("GÉNOVA", "Quetzaltenango"),
    ("FLORES COSTA CUCA", "Quetzaltenango"),
    ("LA ESPERANZA", "Quetzaltenango"),
    ("CAJOLÁ", "Quetzaltenango"),
    ("PALESTINA DE LOS ALTOS", "Quetzaltenango"),
    ("SAN MIGUEL SIGÜILÁ", "Quetzaltenango"),
    ("SAN FRANCISCO LA UNIÓN", "Quiché"),
    ("SAN CARLOS SIJA", "Quetzaltenango"),
    ("RETALHULEU", "Retalhuleu"),
    ("SAN SEBASTIÁN", "Quetzaltenango"),
    ("SAN MARTÍN ZAPOTITLÁN", "Retalhuleu"),
    ("SAN FELIPE", "Retalhuleu"),
    ("SAN ANDRÉS VILLA SECA", "Suchitepéquez"),
    ("CHAPERICO", "Retalhuleu"),
    ("NUEVO SAN CARLOS", "Retalhuleu"),
    ("EL ASINTAL", "Retalhuleu"),
    ("SANTA CRUZ MULUÁ", "Retalhuleu"),
    ("SAN MARCOS", "San Marcos"),
    ("SAN ANTONIO SACATEPÉQUEZ", "San Marcos"),
    ("COMITANCILLO", "San Marcos"),
    ("SAN MIGUEL IXTAHUACÁN", "San Marcos"),
    ("CONCEPCIÓN TUTUAPA", "San Marcos"),
    ("TACANÁ", "San Marcos"),
    ("SIBINAL", "San Marcos"),
    ("TAJUMULCO", "San Marcos"),
    ("TEJUTLA", "San Marcos"),
    ("SAN RAFAEL PIÉ DE LA CUESTA", "San Marcos"),
    ("EL TUMBADOR", "San Marcos"),
    ("EL RODEO", "San Marcos"),
    ("MALACATÁN", "San Marcos"),
    ("OCÓS", "San Marcos"),
    ("SAN PABLO", "San Marcos"),
    ("EL QUETZAL", "San Marcos"),
    ("LA REFORMA", "San Marcos"),
    ("PAJAPITA", "San Marcos"),
    ("IXCHIGUÁN", "San Marcos"),
    ("SIPACAPA", "San Marcos"),
    ("ESQUIPULAS PALO GORDO", "San Marcos"),
    ("RÍO BLANCO", "San Marcos"),
    ("SAN LORENZO", "San Marcos"),
    ("LA BLANCA", "San Marcos"),
    ("NUEVO PROGRESO", "San Marcos"),
    ("AYUTLA", "San Marcos"),
    ("CATARINA", "San Marcos"),
    ("SAN JOSÉ OJETENAM", "San Marcos"),
    ("SAN CRISTÓBAL CUCHO", "San Marcos"),
    ("HUEHUETENANGO", "Huehuetenango"),
    ("MALACATANCITO", "Huehuetenango"),
    ("CUILCO", "Huehuetenango"),
    ("SAN PEDRO NECTA", "Huehuetenango"),
    ("SOLOMA", "Huehuetenango"),
    ("SANTA BÁRBARA", "Huehuetenango"),
    ("LA LIBERTAD", "Huehuetenango"),
    ("SAN MIGUEL ACATÁN", "Huehuetenango"),
    ("SAN RAFAEL LA INDEPENDENCIA", "Huehuetenango"),
    ("TODOS SANTOS CUCHUMATÁN", "Huehuetenango"),
    ("SANTA EULALIA", "Huehuetenango"),
    ("TECTITÁN", "Huehuetenango"),
    ("SAN JUAN IXCOY", "Huehuetenango"),
    ("SAN ANTONIO HUISTA", "Huehuetenango"),
    ("SAN SEBASTIÁN COATÁN", "Huehuetenango"),
    ("AGUACATÁN", "Huehuetenango"),
    ("SAN GASPAR IXCHIL", "Huehuetenango"),
    ("SAN JUAN ATITÁN", "Huehuetenango"),
    ("NENTÓN", "Huehuetenango"),
    ("JACALTENANGO", "Huehuetenango"),
    ("IXTAHUCÁN", "Huehuetenango"),
    ("SAN RAFAEL PETZAL", "Huehuetenango"),
    ("PETATAN", "Huehuetenango"),
    ("SAN SEBASTIÁN HUEHUETENANGO", "Huehuetenango"),
    ("SANTIAGO CHIMALTENANGO", "Huehuetenango"),
    ("SANTA ANA HUISTA", "Huehuetenango"),
    ("BARILLAS", "Huehuetenango"),
    ("CONCEPCIÓN HUISTA", "Huehuetenango"),
    ("SANTA CRUZ DEL QUICHÉ", "Quiché"),
    ("ZACUALPA", "Quiché"),
    ("CHAJUL", "Quiché"),
    ("CHICHICASTENANGO", "Quiché"),
    ("PATZITÉ", "Quiché"),
    ("SAN ANTONIO ILOTENANGO", "Quiché"),
    ("SAN PEDRO JOCOPILAS", "Quiché"),
    ("CUNÉN", "Quiché"),
    ("SAN JUAN COTZAL", "Quiché"),
    ("JOYABAJ", "Quiché"),
    ("NEBAJ", "Quiché"),
    ("SAN ANDRÉS SAJCABAJÁ", "Quiché"),
    ("SAN BARTALOMÉ JOCOTENANGO", "Quiché"),
    ("CANILLÁ", "Quiché"),
    ("CHICAMÁN", "Quiché"),
    ("IXCÁN", "Quiché"),
    ("PACHALUM", "Quiché"),
    ("CHICHÉ", "Quiché"),
    ("CHINIQUE", "Quiché"),
    ("USPANTÁN", "Quiché"),
    ("SACAPULAS", "Quiché"),
    ("SALAMÁ", "Baja Verapaz"),
    ("SAN MIGUEL CHICAJ", "Baja Verapaz"),
    ("RABINAL", "Baja Verapaz"),
    ("CUBULCO", "Baja Verapaz"),
    ("GRANADOS", "Baja Verapaz"),
    ("SAN JERÓNIMO", "Baja Verapaz"),
    ("PURULHÁ", "Baja Verapaz"),
    ("EL CHOL", "Baja Verapaz"),
    ("COBÁN", "Alta Verapaz"),
    ("SANTA CRUZ VERAPAZ", "Alta Verapaz"),
    ("SAN CRISTÓBAL VERAPAZ", "Alta Verapaz"),
    ("TACTIC", "Alta Verapaz"),
    ("TAMAHÚ", "Alta Verapaz"),
    ("TUCURÚ", "Alta Verapaz"),
    ("PANZÓS", "Alta Verapaz"),
    ("SENAHÚ", "Alta Verapaz"),
    ("SAN PEDRO CARCHÁ", "Alta Verapaz"),
    ("SAN JUAN CHAMELCO", "Alta Verapaz"),
    ("LANQUÍN", "Alta Verapaz"),
    ("CHISEC", "Alta Verapaz"),
    ("CHAHAL", "Alta Verapaz"),
    ("FRAY BARTOLOMÉ DE LAS CASAS", "Alta Verapaz"),
    ("SAN CATALINA LA TINTA", "Alta Verapaz"),
    ("RAXRUJÀ", "Alta Verapaz"),
    ("CAHABÓN", "Alta Verapaz"),
    ("FLORES", "Petén"),
    ("SAN ANDRÉS", "Petén"),
    ("SAN FRANCISCO", "Petén"),
    ("DOLORES", "Petén"),
    ("SAN LUIS", "Petén"),
    ("SAYAXCHÉ", "Petén"),
    ("MELCHOR DE MENCOS", "Petén"),
    ("POPTÚN", "Petén"),
    ("LAS CRUCES", "Petén"),
    ("EL CHAL", "Petén"),
    ("SAN BENITO", "Petén"),
    ("SANTA ANA", "Petén"),
    ("GUASTATOYA", "El Progreso"),
    ("MORAZÁN", "El Progreso"),
    ("SAN AGUSTÍN ACASAGUASTLAN", "El Progreso"),
    ("SAN CRISTÓBAL ACASAGUASTLAN", "El Progreso"),
    ("EL JÍCARO", "El Progreso"),
    ("SANSARE", "El Progreso"),
    ("SANARATE", "El Progreso"),
    ("SAN ANTONIO LA PAZ", "El Progreso"),
    ("ANTIGUA GUATEMALA", "Sacatepéquez"),
    ("JOCOTENANGO", "Sacatepéquez"),
    ("PASTORES", "Sacatepéquez"),
    ("SUMPANGO", "Sacatepéquez"),
    ("SANTO DOMINGO XENACOJ", "Sacatepéquez"),
    ("SANTIAGO SACATEPÉQUEZ", "Sacatepéquez"),
    ("SAN BARTOLOMÉ MILPAS ALTAS", "Sacatepéquez"),
    ("SAN LUCAS SACATEPÉQUEZ", "Sacatepéquez"),
    ("SANTA LUCÍA MILPAS ALTAS", "Sacatepéquez"),
    ("MAGDALENA MILPAS ALTAS", "Sacatepéquez"),
    ("SANTA MARÍA DE JESÚS", "Sacatepéquez"),
    ("CIUDAD VIEJA", "Sacatepéquez"),
    ("SAN MIGUEL DUEÑAS", "Sacatepéquez"),
    ("ALOTENANGO", "Sacatepéquez"),
    ("SANTA CATARINA BARAHONA", "Sacatepéquez"),
    ("MAZATENANGO", "Suchitepéquez"),
    ("CUYOTENANGO", "Suchitepéquez"),
    ("SAN FRANCISCO ZAPOTITLÁN", "Suchitepéquez"),
    ("SAN BERNARDINO", "Suchitepéquez"),
    ("SAN JOSÉ EL ÍDOLO", "Suchitepéquez"),
    ("SANTO DOMINGO SUCHITEPÉQUEZ", "Suchitepéquez"),
    ("SAMAYAC", "Suchitepéquez"),
    ("SAN PABLO JOCOPILAS", "Suchitepéquez"),
    ("SAN ANTONIO SUCHITEPÉQUEZ", "Suchitepéquez"),
    ("SAN GABRIEL", "Suchitepéquez"),
    ("CHICACAO", "Suchitepéquez"),
    ("SAN JUAN BAUTISTA", "Suchitepéquez"),
    ("SANTO TOMÁS LA UNIÓN", "Suchitepéquez"),
    ("ZUNILITO", "Suchitepéquez"),
    ("PUEBLO NUEVO", "Suchitepéquez"),
    ("RÍO BRAVO", "Suchitepéquez"),
    ("SAN JOSE LA MÁQUINA", "Suchitepéquez"),
    ("PUERTO BARRIOS", "Izabal"),
    ("LIVINGSTON", "Izabal"),
    ("EL ESTOR", "Izabal"),
    ("MORALES", "Izabal"),
    ("ZACAPA", "Zacapa"),
    ("RÍO HONDO", "Zacapa"),
    ("GUALÁN", "Zacapa"),
    ("TECULUTÁN", "Zacapa"),
    ("SAN DIEGO", "Zacapa"),
    ("HUITÉ", "Zacapa"),
    ("CHIQUIMULA", "Chiquimula"),
    ("SAN JUAN ERMITA", "Chiquimula"),
    ("ESQUIPULAS", "Chiquimula"),
    ("CONCEPCIÓN LAS MINAS", "Chiquimula"),
    ("QUETZALTEPEQUE", "Chiquimula"),
    ("SAN JACINTO", "Chiquimula"),
    ("IPALA", "Chiquimula"),
    ("JALAPA", "Jalapa"),
    ("SAN PEDRO PINULA", "Jalapa"),
    ("SAN MANUEL CHAPARRÓN", "Jalapa"),
    ("MONJAS", "Jalapa"),
    ("MATAQUESCUINTLA", "Jalapa"),
    ("JUTIAPA", "Jutiapa"),
    ("EL PROGRESO", "Jutiapa"),
    ("SANTA CATARINA MITA", "Jutiapa"),
    ("AGUA BLANCA", "Jutiapa"),
    ("ASUNCIÓN MITA", "Jutiapa"),
    ("YUPILTEPEQUE", "Jutiapa"),
    ("ATESCATEMPA", "Jutiapa"),
    ("JEREZ", "Jutiapa"),
    ("ZAPOTITLÁN", "Jutiapa"),
    ("JALPATAGUA", "Jutiapa"),
    ("PASACO", "Jutiapa"),
    ("QUEZADA", "Jutiapa"),
    ("SAN ANTONIO AGUAS CALIENTES", "Sacatepéquez"),
    ("SAN MIGUEL PANÁN", "Jutiapa"),
    ("LOS AMATES", "Izabal"),
    ("USUMATLÁN", "Zacapa"),
    ("LA UNIÓN", "Zacapa"),
    ("OLOPA", "Chiquimula"),
    ("EL ADELANTO", "Jutiapa"),
    ("COMAPA", "Jutiapa"),
    ("SAN JOSÉ ACATEMPA", "Jutiapa"),
    ("PATULUL", "Suchitepéquez"),
    ("CABAÑAS", "Zacapa"),
    ("SAN JOSÉ LA ARADA", "Chiquimula"),
    ("SAN LUIS JILITEPEQUE", "Jalapa"),
    ("MOYUTA", "Jutiapa"),
    ("CONGUACO", "Jutiapa"),
    ("ESTANZUELA", "Zacapa"),
    ("SAN JORGE", "Zacapa"),
    ("JOCOTÁN", "Chiquimula"),
    ("CAMOTÁN", "Chiquimula"),
    ("LA DEMOCRACIA", "Huehuetenango"),
    ("CHIANTLA", "Huehuetenango"),
    ("SAN MATEO IXTATÁN", "Huehuetenango"),
    ("ZUNIL", "Quetzaltenango"),
    ("La Tinta", "Alta Verapaz"),
    ("Fray Bartolomé de las Casas", "Alta Verapaz"),
    ("Guanazapa", "Escuintla"),
    ("San Joseé La Máquina", "Suchitepéquez"),
    ("San Cristobal Acasaguastlán", "El Progreso"),
    ("Santa Lucia Utatlán", "Sololá"),
    ("San Antonio la Paz", "El Progreso"),
    ("San Francisco la Unión", "Quiché"),
    ("San Francisco el Alto", "Totonicapán"),
    ("Santiago Sactepéquez", "Sacatepéquez"),
    ("Santa Catalina Ixtahuacán", "Sololá"),
    ("Santo Tomás la Unión", "Suchitepéquez"),
    ("San Andres Semetabaj", "Sololá"),
    ("Melchor", "Petén"),
    ("San Rafael la Independencia", "Huehuetenango"),
    ("El Jícaro", "El Progreso"),
    ("Santiago Chimaltenanango", "Huehuetenango"),
    ("Génova", "Quetzaltenango"),
    ("San Luis", "Petén"),
    ("Sanarate", "El Progreso"),
    ("Pachalum", "Quiché"),
    ("Santa Apolonia", "Chimaltenango"),
    ("San José Ojetenam", "San Marcos"),
    ("Zacapa", "Zacapa"),
    ("San Cristóbal Cucho", "San Marcos"),
    ("Jalpatagua", "Jutiapa"),
    ("San Lucas Sacatepéquez", "Sacatepéquez"),
    ("Nuevo Progreso", "San Marcos"),
    ("San Bartolomé Milpas Altas", "Sacatepéquez"),
    ("Petatán", "Huehuetenango"),
    ("El Progreso", "Jutiapa"),
    ("LA DEMOCARACIA", "Escuintla"),
    ("Pueblo Nuevo", "Suchitepequez"),
    ("RAXUJÀ", "Alta Verapaz"),
    ("HUEHUETENAGO", "Huehuetenango"),
    ("Mazatenango", "Suchitepéquez"),
    ("Patulul", "Suchitepéquez"),
    ("San Bernardino", "Suchitepéquez"),
    ("San Miguel Panán", "Suchitepéquez"),
]

In [16]:
municipios = [
    "Guatemala",
"Santa Catarina Pinula",
"San José Pinula",
"San José del Golfo",
"Palencia",
"Chinautla",
"San Pedro Ayampuc",
"Mixco",
"San Pedro Sacatepéquez",
"San Juan Sacatepéquez",
"San Raymundo",
"Chuarrancho",
"Fraijanes",
"Amatitlán",
"Villa Nueva",
"Villa Canales",
"Petapa",
"Guastatoya",
"Morazán",
"San Agustín Acasaguastlán",
"San Cristóbal Acasaguastlán",
"El Jícaro",
"Sansare",
"Sanarate",
"San Antonio La Paz",
"Antigua Guatemala",
"Jocotenango",
"Pastores",
"Sumpango",
"Santo Domingo Xenacoj",
"Santiago Sacatepéquez",
"San Bartolomé Milpas Altas",
"San Lucas Sacatepéquez",
"Santa Lucía Milpas Altas",
"Magdalena Milpas Altas",
"Santa María de Jesús",
"Ciudad Vieja",
"San Miguel Dueñas",
"Alotenango",
"San Antonio Aguas Calientes",
"Santa Catarina Barahona",
"Chimaltenango",
"San José Poaquil",
"San Martín Jilotepeque",
"Comalapa",
"Santa Apolonia",
"Tecpán Guatemala",
"Patzún",
"Pochuta",
"Patzicía",
"Santa Cruz Balanyá",
"Acatenango",
"Yepocapa",
"San Andrés Itzapa",
"Parramos",
"Zaragoza",
"El Tejar",
"Escuintla",
"Santa Lucía Cotzumalguapa",
"La Democracia",
"Siquinalá",
"Masagua",
"Tiquisate",
"La Gomera",
"Guanagazapa",
"San José",
"Iztapa",
"Palín",
"San Vicente Pacaya",
"Nueva Concepción",
"Sipacate",
"Cuilapa",
"Barberena",
"Santa Rosa de Lima",
"Casillas",
"San Rafael Las Flores",
"Oratorio",
"San Juan Tecuaco",
"Chiquimulilla",
"Taxisco",
"Santa María Ixhuatán",
"Guazacapán",
"Santa Cruz Naranjo",
"Pueblo Nuevo Viñas",
"Nueva Santa Rosa",
"Sololá",
"San José Chacayá",
"Santa María Visitación",
"Santa Lucía Utatlán",
"Nahualá",
"Santa Catarina Ixtahuacán",
"Santa Clara La Laguna",
"Concepción",
"San Andrés Semetabaj",
"Panajachel",
"Santa Catarina Palopó",
"San Antonio Palopó",
"San Lucas Toliman",
"Santa Cruz La Laguna",
"San Pablo La Laguna",
"San Marcos La Laguna",
"San Juan La Laguna",
"San Pedro La Laguna",
"Santiago Atitlán",
"Totonicapán",
"San Cristóbal Totonicapán",
"San Francisco El Alto",
"San Andrés Xecul",
"Momostenango",
"Santa María Chiquimula",
"Santa Lucía La Reforma",
"San Bartolo",
"Quetzaltenango",
"Salcajá",
"Olintepeque",
"San Carlos Sija",
"Sibilia",
"Cabricán",
"Cajolá",
"San Miguel Siguilá",
"Ostuncalco",
"San Mateo",
"Concepción Chiquirichapa",
"San Martín Sacatepéquez",
"Almolonga",
"Cantel",
"Huitán",
"Zunil",
"Colomba",
"San Francisco La Unión",
"El Palmar",
"Coatepeque",
"Génova",
"Flores Costa Cuca",
"La Esperanza",
"Palestina de Los Altos","Mazatenango",
"Cuyotenango",
"San Francisco Zapotitlán",
"San Bernardino",
"San José El Ídolo",
"Santo Domingo Suchitepéquez",
"San Lorenzo",
"Samayac",
"San Pablo Jocopilas",
"San Antonio Suchitepéquez",
"San Miguel Panán",
"San Gabriel",
"Chicacao",
"Patulul",
"Santa Bárbara",
"San Juan Bautista",
"Santo Tomás La Unión",
"Zunilito",
"Pueblo Nuevo ",
"Rio Bravo",
"San José La Máquina",
"Retalhuleu",
"San Sebastián",
"Santa Cruz Muluá",
"San Martín Zapotitlán",
"San Felipe",
"San Andrés Villa Seca",
"Champerico",
"Nuevo San Carlos",
"El Asintal",
"San Marcos",
"San Pedro Sacatepéquez",
"San Antonio Sacatepéquez",
"Comitancillo",
"San Miguel Ixtahuacán",
"Concepción Tutuapa",
"Tacaná",
"Sibinal",
"Tajumulco",
"Tejutla",
"San Rafael Pie de La Cuesta",
"Nuevo Progreso",
"El Tumbador",
"El Rodeo",
"Malacatán",
"Catarina",
"Ayutla",
"Ocós",
"San Pablo",
"El Quetzal",
"La Reforma",
"Pajapita",
"Ixchiguán",
"San José Ojetenam",
"San Cristóbal Cucho",
"Sipacapa",
"Esquipulas Palo Gordo",
"Rio Blanco",
"San Lorenzo",
"La Blanca",
"Huehuetenango",
"Chiantla",
"Malacatancito",
"Cuilco",
"Nentón",
"San Pedro Necta",
"Jacaltenango",
"Soloma",
"Ixtahuacán",
"Santa Bárbara",
"La Libertad",
"La Democracia",
"San Miguel Acatán",
"San Rafael La Independencia",
"Todos Santos Cuchumatán",
"San Juan Atitán",
"Santa Eulalia",
"San Mateo Ixtatán",
"Colotenango",
"San Sebastián Huehuetenango",
"Tectitán",
"Concepción Huista",
"San Juan Ixcoy",
"San Antonio Huista",
"San Sebastián Coatán",
"Barillas",
"Aguacatán",
"San Rafael Petzal",
"San Gaspar Ixchil",
"Santiago Chimaltenango",
"Santa Ana Huista",
"Unión Cantinil",
"Petatán",
"Santa Cruz del Quiché",
"Chiché",
"Chinique",
"Zacualpa",
"Chajul",
"Chichicastenango",
"Patzité",
"San Antonio Ilotenango",
"San Pedro Jocopilas",
"Cunén",
"San Juan Cotzal",
"Joyabaj",
"Nebaj",
"San Andrés Sajcabajá",
"Uspantán",
"Sacapulas",
"San Bartolomé Jocotenango",
"Canillá",
"Chicamán",
"Ixcán",
"Pachalum",
"Salamá",
"San Miguel Chicaj",
"Rabinal",
"Cubulco",
"Granados",
"El Chol",
"San Jerónimo",
"Purulhá",
"Cobán",
"Santa Cruz Verapaz",
"San Cristóbal Verapaz",
"Tactic",
"Tamahú",
"Tucurú",
"Panzós",
"Senahú",
"San Pedro Carchá",
"San Juan Chamelco",
"Lanquín",
"Cahabón",
"Chisec",
"Chahal",
"Fray Bartolomé de Las Casas",
"Santa Catalina la Tinta",
"Raxruhá",
"Flores",
"San José",
"San Benito",
"San Andrés",
"La Libertad",
"San Francisco",
"Santa Ana",
"Dolores",
"San Luis",
"Sayaxché",
"Melchor de Mencos",
"Poptún",
"Las Cruces",
"El Chal",
"Puerto Barrios",
"Livingston",
"El Estor",
"Morales",
"Los Amates",
"Zacapa",
"Estanzuela",
"Río Hondo",
"Gualán",
"Teculután",
"Usumatlán",
"Cabañas",
"San Diego",
"La Unión",
"Huité",
"San Jorge",
"Chiquimula",
"San José La Arada",
"San Juan Erminta",
"Jocotán",
"Camotán",
"Olopa",
"Esquipulas",
"Concepción Las Minas",
"Quetzaltepeque",
"San Jacinto",
"Ipala",
"Jalapa",
"San Pedro Pinula",
"San Luis Jilotepeque",
"San Manuel Chaparrón",
"San Carlos Alzatate",
"Monjas",
"Mataquescuintla",
"Jutiapa",
"El Progreso",
"Santa Catarina Mita",
"Agua Blanca",
"Asunción Mita",
"Yupiltepeque",
"Atescatempa",
"Jerez",
"El Adelanto",
"Zapotitlán",
"Comapa",
"Jalpatagua",
"Conguaco",
"Moyuta",
"Pasaco",
"San José Acatempa",
"Quesada",
].extend(pd.DataFrame(municipios_departamentos)[0].to_list())

In [17]:
# Normalización de municipios y creación de lista de tuplas (municipio, departamento)
import unicodedata
def normalize_text(text):
    text = str(text).strip().lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return text
df["MUNICIPIO_NORM"] = df["Municipio"].apply(normalize_text)
df["DEPARTAMENTO_NORM"] = df["DEPARTAMENTO"].apply(normalize_text)
municipios_departamentos = list(zip(df["MUNICIPIO_NORM"], df["DEPARTAMENTO_NORM"]))
municipios_departamentos[:10]  # Muestra los primeros 10 para verificar

[('jocotenango', 'sacatepequez'),
 ('chichicastenango', 'quiche'),
 ('coatepeque', 'quetzaltenango'),
 ('camanchaj', 'quiche'),
 ('san pedro sac. s.m.', 'san marcos'),
 ('totonicapan', 'totonicapan'),
 ('san miguel duenas', 'sacatepequez'),
 ('san andres itzapa', 'chimaltenango'),
 ('el tejar', 'chimaltenango'),
 ('rio hondo', 'zacapa')]

In [18]:
df["MUNICIPIO_NORM"]

DIRECCIÓN
2                       jocotenango
3                  chichicastenango
4                        coatepeque
5                         camanchaj
6               san pedro sac. s.m.
                   ...             
130            amatitlan, guatemala
131    santa catarina mita, jutiapa
132                      san marcos
133             santa maria cahabon
134             san jose la maquina
Name: MUNICIPIO_NORM, Length: 133, dtype: object

In [19]:
# Supón que tienes una lista oficial de tuplas (municipio, departamento) llamada municipios_departamentos_oficial
# Ejemplo: municipios_departamentos_oficial = [("guatemala", "guatemala"), ("mixco", "guatemala"), ...]
from difflib import get_close_matches
def match_municipio(muni, muni_list):
    match = get_close_matches(muni, muni_list, n=1, cutoff=0.8)
    return match[0] if match else None

# Extrae la lista de municipios oficiales normalizados
municipios_oficiales = [normalize_text(m) for m, d in municipios_departamentos]
df["MUNICIPIO_NORM_MATCH"] = df["MUNICIPIO_NORM"].apply(lambda x: match_municipio(x, municipios_oficiales))

In [20]:
df["MUNICIPIO_NORM_MATCH"] = df["Municipio"].apply(lambda x: match_municipio(x, municipios_oficiales))

In [21]:
df[df["MUNICIPIO_NORM_MATCH"].isnull()]["Municipio"].shape

(25,)

In [22]:
df["MUNICIPIO_NORM_MATCH"].value_counts().sort_values(ascending=False)

df["DEPARTAMENTO_NORM"].value_counts().sort_values(ascending=False)

DEPARTAMENTO_NORM
san marcos        19
sacatepequez      13
chimaltenango     13
guatemala         13
suchitepequez     10
quiche             9
jutiapa            9
santa rosa         8
quetzaltenango     7
alta verapaz       7
el progreso        4
zacapa             4
escuintla          3
peten              3
totonicapan        2
jalapa             2
solola             2
retalhuleu         2
huehuetenango      1
chiquimula         1
izabal             1
Name: count, dtype: int64

In [23]:
df["MUNICIPIO_NORM_MATCH"].loc[df["MUNICIPIO_NORM_MATCH"].isnull()] = df[df["MUNICIPIO_NORM_MATCH"].isnull()]["MUNICIPIO_NORM"]

/tmp/ipykernel_117398/12905116.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["MUNICIPIO_NORM_MATCH"].loc[df["MUNICIPIO_NORM_MATCH"].isnull()] = df[df["MUNICIPIO_NORM_MATCH"].isnull()]["MUNICIPIO_NORM"]


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 133 entries, 2 to 134
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Municipio             133 non-null    object
 1   DIRECCIÓN.1           133 non-null    object
 2   DEPARTAMENTO          133 non-null    object
 3   TELÉFONO              133 non-null    object
 4   CORREO                132 non-null    object
 5   DEPARTAMENTO_LIMPIO   133 non-null    object
 6   MUNICIPIO_NORM        133 non-null    object
 7   DEPARTAMENTO_NORM     133 non-null    object
 8   MUNICIPIO_NORM_MATCH  133 non-null    object
dtypes: object(9)
memory usage: 10.4+ KB


In [25]:
df["MUNICIPIO_NORM_MATCH"].value_counts()
df["DEPARTAMENTO_NORM"].value_counts()

DEPARTAMENTO_NORM
san marcos        19
sacatepequez      13
chimaltenango     13
guatemala         13
suchitepequez     10
quiche             9
jutiapa            9
santa rosa         8
quetzaltenango     7
alta verapaz       7
el progreso        4
zacapa             4
escuintla          3
peten              3
totonicapan        2
jalapa             2
solola             2
retalhuleu         2
huehuetenango      1
chiquimula         1
izabal             1
Name: count, dtype: int64

In [26]:
grouped_municipio = df.\
    groupby(["MUNICIPIO_NORM_MATCH", "DEPARTAMENTO_NORM"]).\
    size().reset_index(name="count")
grouped_departamento = df.\
    groupby(["DEPARTAMENTO_NORM"]).\
    size().reset_index(name="count")
df = grouped_municipio.\
    merge(
        grouped_departamento, 
        on="DEPARTAMENTO_NORM", 
        how="outer", 
        suffixes=("_municipio", "_departamento")
    )

## Unificar con los valores de las variables de CBM

In [27]:
# Elimina columnas duplicadas si existen
df = df.loc[:, ~df.columns.duplicated()]

# Sumarle las 22 estaciones de bomberos por parte del CBM
if df[df["MUNICIPIO_NORM_MATCH"]=="guatemala"].shape[0]>0:
    df.loc[df["MUNICIPIO_NORM_MATCH"]=="guatemala", "count_municipio"] += 22
else:
    row = pd.Series({"DEPARTAMENTO_NORM": "guatemala", "MUNICIPIO_NORM_MATCH": "guatemala", "count_municipio": 22, "count_departamento": df[df["DEPARTAMENTO_NORM"]=="guatemala"]["count_departamento"].values[0] if df[df["DEPARTAMENTO_NORM"]=="guatemala"].shape[0]>0 else 0})
    df = pd.concat([df, pd.DataFrame([row])], axis=0)
df.loc[df["DEPARTAMENTO_NORM"]=="guatemala", "count_departamento"] += 22


In [28]:
mapping = {
    # Alta Verapaz
    "tactic alta verapaz": "Tactic",
    "santa maria cahabon": "Cahabón",
    "san miguel tucuru": "Tucurú",

    # Chiquimula
    "ipala chiquimulilla": "Ipala",

    # Guatemala
    "aldea el fiscal palencia": "Palencia",
    "santa isabel villa nueva": "Villa Nueva",

    # Escuintla
    "masagua masagua escuintla": "Masagua",
    "san jose escuintla": "San José",

    # Jutiapa
    "ciudad pedro de alvarado moyuta jutiapa": "Moyuta",

    # San Marcos
    "san pablo san marcos": "San Pablo",
    "aldea el tocache san pablo san marcos": "San Pablo",
    "san antonio sac s m": "San Antonio Sacatepéquez",
    "san pedro sac s m": "San Pedro Sacatepéquez",
    "san jose el rodeo s a": "El Rodeo",

    # Santa Rosa
    "sector playa chuquimulilla": "Chiquimulilla",

    # Sacatepéquez
    "magdalena m a": "Magdalena Milpas Altas",
    "san bartolome m a": "San Bartolomé Milpas Altas",
    "san antonio aguas c": "San Antonio Aguas Calientes",

    # Chimaltenango
    "santa cruz balanya": "Santa Cruz Balanyá",
}


In [29]:
import pandas as pd
from rapidfuzz import process

mapping = {
    "tactic alta verapaz": "Tactic",
    "santa maria cahabon": "Cahabón",
    "san miguel tucuru": "Tucurú",
    "ipala chiquimulilla": "Ipala",
    "aldea el fiscal palencia": "Palencia",
    "santa isabel villa nueva": "Villa Nueva",
    "masagua masagua escuintla": "Masagua",
    "san jose escuintla": "San José",
    "ciudad pedro de alvarado moyuta jutiapa": "Moyuta",
    "san pablo san marcos": "San Pablo",
    "aldea el tocache san pablo san marcos": "San Pablo",
    "san antonio sac s m": "San Antonio Sacatepéquez",
    "san pedro sac s m": "San Pedro Sacatepéquez",
    "san jose el rodeo s a": "El Rodeo",
    "sector playa chuquimulilla": "Chiquimulilla",
    "magdalena m a": "Magdalena Milpas Altas",
    "san bartolome m a": "San Bartolomé Milpas Altas",
    "san antonio aguas c": "San Antonio Aguas Calientes",
    "santa cruz balanya": "Santa Cruz Balanyá",
}

# Lista oficial de municipios (ejemplo, deberías tener la tuya completa)
municipios_oficiales = [
    "Tactic", "Cahabón", "Tucurú", "Ipala", "Palencia", "Villa Nueva", 
    "Masagua", "San José", "Moyuta", "San Pablo", "San Antonio Sacatepéquez", 
    "San Pedro Sacatepéquez", "El Rodeo", "Chiquimulilla", 
    "Magdalena Milpas Altas", "San Bartolomé Milpas Altas", 
    "San Antonio Aguas Calientes", "Santa Cruz Balanyá"
]

# Función de normalización
def normalizar(texto):
    return str(texto).strip().lower()

# Función para estandarizar
def estandarizar(nombre):
    clave = normalizar(nombre)
    if clave in mapping:
        return mapping[clave]
    result = process.extractOne(clave, municipios_oficiales, score_cutoff=85)
    if result is not None:
        mejor, score, _ = result
        return mejor
    else:
        return nombre

# Crear la nueva columna
df["MUNICIPIO_NORM_MATCH"] = df["MUNICIPIO_NORM_MATCH"].apply(estandarizar)


In [30]:
df["MUNICIPIO_NORM_MATCH"]

0      fray bartolome de las casas
1                         la tinta
2                san juan chamelco
3                           Tucurú
4                          Cahabón
                  ...             
129                          huite
130                la union zacapa
131                      rio hondo
132                      usumatlan
0                        guatemala
Name: MUNICIPIO_NORM_MATCH, Length: 134, dtype: object

## Generar el archivo que será útil para unificar

In [ ]:
df.to_csv(
        f"./dataset/variables_alternas_procesadas/distribucion_bomberos_departamentales.csv", 
        index=False
    )